# Figure 4 复现：时序注意力可视化

本 notebook 复现论文中的 Figure 4，展示自回归视频生成过程中注意力权重在帧间的分布。

**论文的关键发现：**
> "与传统理解（认为只需保留少量初始 KV token）不同，我们的分析揭示：
> 大多数注意力头不仅对最早的 token 分配了大量权重，还对序列中间部分分配了相当的注意力。"

**Figure 4 描述：**
> "Query 平均注意力展示了最后一个 chunk（第 19-21 帧）如何关注之前的 KV cache 条目（第 0-18 帧）。
> 我们可视化了来自不同层的两个代表性注意力头——L1H1（第 1 层第 1 个头）和 L5H10（第 5 层第 10 个头）——
> 表明注意力在整个上下文窗口中都保持显著，而不仅仅集中在初始帧。"

## 前置条件

首先运行提取脚本：
```bash
python run_extraction_figure4.py \
    --config_path configs/self_forcing_dmd.yaml \
    --checkpoint_path checkpoints/self_forcing_dmd.pt \
    --output_path attention_cache_figure4.pt \
    --layer_indices 0 4
```

**注意**：由于 KV cache 推理模式的滑动窗口限制，捕获的数据可能只包含部分 token。
如需完整的帧级注意力，请使用 `run_extraction_figure4.py` 脚本。

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# 设置绘图风格
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")

# 高质量 SVG 输出配置
plt.rcParams.update({
    'svg.fonttype': 'none',           # 使用真实字体而非路径
    'font.family': 'sans-serif',
    'font.sans-serif': ['Arial', 'DejaVu Sans', 'Helvetica'],
    'font.size': 11,
    'axes.labelsize': 12,
    'axes.titlesize': 13,
    'xtick.labelsize': 10,
    'ytick.labelsize': 10,
    'legend.fontsize': 10,
    'figure.dpi': 150,
    'savefig.dpi': 300,               # 高质量输出
    'savefig.bbox': 'tight',
    'savefig.pad_inches': 0.1,
})

## 1. 加载注意力权重

加载由 `run_extraction_figure4.py` 生成的注意力数据缓存。

In [ ]:
# 加载注意力权重缓存
# 支持两种格式：
# 1. run_extraction_figure4.py 生成的完整注意力数据（推荐）
# 2. run_extraction.py 生成的推理模式数据

CACHE_PATH = "attention_cache_figure4.pt"  # 优先使用完整数据
if not Path(CACHE_PATH).exists():
    CACHE_PATH = "attention_cache.pt"  # 回退到推理模式数据

if not Path(CACHE_PATH).exists():
    raise FileNotFoundError(
        f"未找到注意力缓存文件。请先运行以下命令之一：\n"
        "  python run_extraction_figure4.py --layer_indices 0 4\n"
        "  python run_extraction.py --layer_indices 0 1 4 5"
    )

# 使用 weights_only=False 因为缓存可能包含 omegaconf 对象
data = torch.load(CACHE_PATH, map_location='cpu', weights_only=False)

print(f"加载注意力数据来源: {CACHE_PATH}")
print("=" * 60)
print(f"  - Prompt: {data.get('prompt', 'N/A')}")
print(f"  - 帧数: {data.get('num_frames', 'N/A')}")
print(f"  - 每帧 token 数: {data.get('frame_seq_length', 'N/A')}")
print(f"  - 每 block 帧数: {data.get('num_frame_per_block', 'N/A')}")
print(f"  - 捕获的层索引: {data.get('layer_indices', 'N/A')}")
print(f"  - 注意力张量数量: {len(data['attention_weights'])}")
print("=" * 60)

# 存储供后续使用
attention_weights = data['attention_weights']
frame_seq_length = data.get('frame_seq_length', 1560)
num_frames = data.get('num_frames', 21)
num_frame_per_block = data.get('num_frame_per_block', 3)

In [ ]:
# 检查捕获的注意力权重详情
print("\n捕获的注意力权重张量：")
print("=" * 80)
for i, w in enumerate(attention_weights):
    layer_idx = w['layer_idx']
    attn_shape = w['attn_weights'].shape
    lq, lk = attn_shape[2], attn_shape[3]
    num_heads = attn_shape[1]

    # 判断数据类型
    if 'last_block_start_frame' in w:
        # 来自 run_extraction_figure4.py 的完整数据
        last_start = w['last_block_start_frame']
        q_frames = (lq // w['frame_seq_length'])
        k_frames = lk // w['frame_seq_length']
        data_type = f"完整注意力 (query: 帧{last_start}-{last_start + q_frames - 1} -> key: 帧0-{k_frames - 1})"
    else:
        # 来自 run_extraction.py 的推理模式数据
        q_frames = lq // frame_seq_length if lq >= frame_seq_length else 0
        k_frames = lk // frame_seq_length if lk >= frame_seq_length else 0

        if lk == lq:
            data_type = "Block 内自注意力"
        elif lk < frame_seq_length:
            data_type = f"滑动窗口 KV cache ({lk} tokens)"
        else:
            data_type = f"KV cache 注意力 ({k_frames} 帧)"

    print(f"  [{i}] Layer {layer_idx:2d}: shape={attn_shape}")
    print(f"       类型: {data_type}")

print("=" * 80)

# 检测数据格式
is_figure4_format = any('last_block_start_frame' in w for w in attention_weights)
print(f"\n数据格式: {'Figure 4 完整格式' if is_figure4_format else '推理模式格式'}")

## 2. 处理注意力权重

注意力张量的形状通常是 `(Batch, Heads, Query_Len, Key_Len)`。

对于 Figure 4，我们需要：
1. 选择特定的层和注意力头
2. 对 Query 维度求平均，得到"每个 key 位置的平均注意力"
3. 将 token 位置映射到帧索引

In [ ]:
def compute_frame_attention_distribution(
    attn_data: dict,
    frame_seq_length: int = 1560,
    num_frames: int = 21,
):
    """
    计算 Figure 4 所需的帧间注意力分布。

    对于 Figure 4，我们需要展示最后一个 chunk（query，第 19-21 帧）
    如何关注之前的 KV cache 条目（key，第 0-18 帧）。

    Args:
        attn_data: 单个层的注意力数据字典
        frame_seq_length: 每帧的 token 数
        num_frames: 总帧数

    Returns:
        frame_attention: 形状为 [num_heads, num_key_frames] 的数组，
                        展示每个 head 对每个 key 帧的平均注意力
        key_frame_indices: key 帧的索引数组
    """
    attn_weights = attn_data['attn_weights']
    B, num_heads, lq, lk = attn_weights.shape

    # 检测数据格式
    if 'last_block_start_frame' in attn_data:
        # Figure 4 完整格式
        fsl = attn_data['frame_seq_length']
        num_key_frames = lk // fsl
    else:
        # 推理模式格式
        fsl = frame_seq_length
        num_key_frames = lk // fsl if lk >= fsl else max(1, lk // (fsl // 10))
        if num_key_frames == 0:
            # 滑动窗口情况，使用 token 分组
            tokens_per_group = max(1, lk // 20)
            num_key_frames = lk // tokens_per_group
            fsl = tokens_per_group

    # 提取注意力矩阵 [B, num_heads, Lq, Lk] -> [num_heads, Lq, Lk]
    attn = attn_weights[0].float().numpy()  # 取 batch 0

    # 计算每个 head 对每个 key 帧的平均注意力
    # 对所有 query 位置求平均，得到"每个 key 位置平均获得多少注意力"
    frame_attention = np.zeros((num_heads, num_key_frames))

    for head_idx in range(num_heads):
        head_attn = attn[head_idx]  # [Lq, Lk]

        # 对所有 query 位置求平均
        avg_attn_per_key_token = head_attn.mean(axis=0)  # [Lk]

        # 按 key 帧分组求和（而非平均）以得到每帧的总注意力
        for kf in range(num_key_frames):
            k_start = kf * fsl
            k_end = min((kf + 1) * fsl, lk)
            if k_end > k_start:
                # 对帧内所有 token 的注意力求和，表示该帧获得的总注意力
                frame_attention[head_idx, kf] = avg_attn_per_key_token[k_start:k_end].sum()

    key_frame_indices = np.arange(num_key_frames)

    return frame_attention, key_frame_indices

In [ ]:
# 为所有捕获的层计算注意力分布
layer_attention_data = {}

for i, w in enumerate(attention_weights):
    layer_idx = w['layer_idx']

    frame_attention, key_frame_indices = compute_frame_attention_distribution(
        w, frame_seq_length=frame_seq_length, num_frames=num_frames
    )

    layer_attention_data[layer_idx] = {
        'frame_attention': frame_attention,  # [num_heads, num_frames]
        'key_frame_indices': key_frame_indices,
        'attn_shape': w['attn_weights'].shape,
        'num_heads': w['attn_weights'].shape[1],
        'is_figure4_format': 'last_block_start_frame' in w,
    }

    print(f"Layer {layer_idx}: {len(key_frame_indices)} key 帧, {frame_attention.shape[0]} 个 head")
    print(f"  注意力范围: [{frame_attention.min():.6f}, {frame_attention.max():.6f}]")

print(f"\n共处理 {len(layer_attention_data)} 层的注意力数据")

## 3. 绘制 Figure 4：帧间注意力分布

论文中的 Figure 4 展示了两个代表性注意力头（L1H1 和 L5H10）的注意力分布。

**注意**：图中所有 label 使用英文以确保兼容性和专业性。

In [ ]:
def plot_figure4_paper_style(
    layer_attention_data: dict,
    heads_to_show: list = None,
    save_path: str = None,
    figsize: tuple = (14, 5),
):
    """
    按论文风格复现 Figure 4，展示帧间注意力分布。
    图中 label 全部使用英文。

    论文描述：
    "Query 平均注意力展示了最后一个 chunk（第 19-21 帧）如何关注之前的
    KV cache 条目（第 0-18 帧）。我们可视化了两个代表性注意力头——
    L1H1（第 1 层第 1 个头）和 L5H10（第 5 层第 10 个头）——
    表明注意力在整个上下文窗口中都保持显著，而不仅仅集中在初始帧。"

    Args:
        layer_attention_data: 层索引 -> 注意力数据的字典
        heads_to_show: (layer_idx, head_idx, label) 元组列表
        save_path: SVG 保存路径
        figsize: 图像大小
    """
    # 默认：展示论文中的 L1H1 和 L5H10
    if heads_to_show is None:
        heads_to_show = []
        if 0 in layer_attention_data:
            heads_to_show.append((0, 0, "L1H1"))
        if 4 in layer_attention_data:
            heads_to_show.append((4, 9, "L5H10"))
        # 如果没有这些层，使用可用的层
        if not heads_to_show:
            for layer_idx in sorted(layer_attention_data.keys())[:2]:
                heads_to_show.append((layer_idx, 0, f"L{layer_idx+1}H1"))

    num_plots = len(heads_to_show)
    if num_plots == 0:
        print("No heads to display!")
        return None

    # 使用 2 列布局
    ncols = min(2, num_plots)
    nrows = (num_plots + ncols - 1) // ncols

    fig, axes = plt.subplots(nrows, ncols, figsize=figsize, squeeze=False)
    axes = axes.flatten()

    # 颜色方案
    colors = ['#2E86AB', '#A23B72', '#F18F01', '#C73E1D', '#3B1F2B']

    for idx, (layer_idx, head_idx, label) in enumerate(heads_to_show):
        ax = axes[idx]

        if layer_idx not in layer_attention_data:
            ax.set_visible(False)
            continue

        data = layer_attention_data[layer_idx]
        frame_attention = data['frame_attention']
        key_frame_indices = data['key_frame_indices']

        # 获取指定 head 的注意力
        if head_idx >= frame_attention.shape[0]:
            head_idx = 0

        head_attn = frame_attention[head_idx]
        color = colors[idx % len(colors)]

        # 绘制柱状图
        ax.bar(key_frame_indices, head_attn, color=color, alpha=0.7,
               edgecolor=color, linewidth=0.5)

        # 叠加折线图显示趋势
        ax.plot(key_frame_indices, head_attn, 'o-', color=color,
                linewidth=1.5, markersize=4, alpha=0.9)

        # 英文标签
        ax.set_xlabel('Key Frame Index', fontsize=12)
        ax.set_ylabel('Attention Weight', fontsize=12)
        ax.set_title(f'{label} (Layer {layer_idx+1}, Head {head_idx+1})',
                    fontsize=13, fontweight='bold')

        # 网格
        ax.grid(True, alpha=0.3, linestyle='--')
        ax.set_axisbelow(True)

        # X 轴刻度
        if len(key_frame_indices) <= 25:
            ax.set_xticks(key_frame_indices)
        else:
            step = max(1, len(key_frame_indices) // 10)
            ax.set_xticks(key_frame_indices[::step])

        # 标注最大值
        max_idx = np.argmax(head_attn)
        ax.annotate(f'max@{key_frame_indices[max_idx]}',
                   xy=(key_frame_indices[max_idx], head_attn[max_idx]),
                   xytext=(5, 5), textcoords='offset points',
                   fontsize=9, color='darkgreen', fontweight='bold')

    # 隐藏未使用的子图
    for idx in range(num_plots, len(axes)):
        axes[idx].set_visible(False)

    # 英文标题
    plt.suptitle(
        'Figure 4: Frame-wise Attention Weight Distribution\n'
        '(Query-averaged attention from last chunk to previous frames)',
        fontsize=14, fontweight='bold', y=1.02
    )
    plt.tight_layout()

    # 保存为高质量 SVG
    if save_path:
        svg_path = save_path if save_path.endswith('.svg') else save_path.replace('.png', '.svg')
        plt.savefig(svg_path, format='svg', bbox_inches='tight', 
                    metadata={'Creator': 'Self-Forcing Figure 4 Reproduction'})
        print(f"已保存到 {svg_path}")

    plt.show()
    return fig

In [ ]:
# 绘制 Figure 4 - 论文风格，展示两个代表性 head
# 根据论文：L1H1（第 1 层第 1 个头）和 L5H10（第 5 层第 10 个头）

fig = plot_figure4_paper_style(
    layer_attention_data,
    heads_to_show=[
        (0, 0, "L1H1"),   # Layer 1, Head 1
        (4, 9, "L5H10"),  # Layer 5, Head 10
    ],
    save_path="figure4_reproduction.svg",
    figsize=(12, 4)
)

## 4. 多头分析：热力图

可视化单层所有注意力头的模式，观察它们是否表现出不同的行为。

In [ ]:
def plot_all_heads_heatmap(
    layer_attention_data: dict,
    layer_idx: int,
    save_path: str = None,
    figsize: tuple = (14, 6),
):
    """
    将单层所有 head 的注意力分布绘制为热力图。
    图中 label 全部使用英文。
    """
    if layer_idx not in layer_attention_data:
        print(f"Layer {layer_idx} 数据不存在！")
        return None

    data = layer_attention_data[layer_idx]
    frame_attention = data['frame_attention']  # [num_heads, num_frames]
    key_frame_indices = data['key_frame_indices']
    num_heads = frame_attention.shape[0]

    fig, ax = plt.subplots(figsize=figsize)

    # 绘制热力图
    im = ax.imshow(frame_attention, cmap='viridis', aspect='auto', interpolation='nearest')
    cbar = plt.colorbar(im, ax=ax)
    cbar.set_label('Attention Weight', fontsize=11)

    # 英文标签
    ax.set_xlabel('Key Frame Index', fontsize=12)
    ax.set_ylabel('Attention Head', fontsize=12)
    ax.set_title(f'Layer {layer_idx + 1}: Attention Distribution Across All Heads',
                fontsize=14, fontweight='bold')

    # Y 轴：head 索引
    ax.set_yticks(range(num_heads))
    ax.set_yticklabels([f'H{i+1}' for i in range(num_heads)])

    # X 轴：帧索引
    if len(key_frame_indices) <= 25:
        ax.set_xticks(range(len(key_frame_indices)))
        ax.set_xticklabels(key_frame_indices)
    else:
        step = max(1, len(key_frame_indices) // 10)
        ax.set_xticks(range(0, len(key_frame_indices), step))
        ax.set_xticklabels(key_frame_indices[::step])

    plt.tight_layout()

    # 保存为高质量 SVG
    if save_path:
        svg_path = save_path if save_path.endswith('.svg') else save_path.replace('.png', '.svg')
        plt.savefig(svg_path, format='svg', bbox_inches='tight',
                    metadata={'Creator': 'Self-Forcing Figure 4 Reproduction'})
        print(f"已保存到 {svg_path}")

    plt.show()
    return fig

In [ ]:
# 绘制所有 head 的热力图
# Layer 0 (L1) - 第 1 层
if 0 in layer_attention_data:
    fig = plot_all_heads_heatmap(
        layer_attention_data,
        layer_idx=0,
        save_path="figure4_layer1_all_heads.svg"
    )

# Layer 4 (L5) - 第 5 层
if 4 in layer_attention_data:
    fig = plot_all_heads_heatmap(
        layer_attention_data,
        layer_idx=4,
        save_path="figure4_layer5_all_heads.svg"
    )

## 5. 跨层注意力对比

比较不同层之间 head 平均注意力分布的差异，验证论文的核心发现。

In [ ]:
def plot_attention_comparison(
    layer_attention_data: dict,
    layers_to_compare: list = None,
    save_path: str = None,
    figsize: tuple = (14, 5),
):
    """
    比较不同层的 head 平均注意力分布。
    验证论文的观点：注意力分布在整个上下文窗口中，而不仅仅集中在初始帧。
    图中 label 全部使用英文。
    """
    if layers_to_compare is None:
        layers_to_compare = sorted(layer_attention_data.keys())

    fig, ax = plt.subplots(figsize=figsize)

    colors = plt.cm.viridis(np.linspace(0, 1, len(layers_to_compare)))

    for idx, layer_idx in enumerate(layers_to_compare):
        if layer_idx not in layer_attention_data:
            continue

        data = layer_attention_data[layer_idx]
        frame_attention = data['frame_attention']
        key_frame_indices = data['key_frame_indices']

        # 对所有 head 求平均
        mean_attention = frame_attention.mean(axis=0)
        std_attention = frame_attention.std(axis=0)

        color = colors[idx]
        ax.plot(key_frame_indices, mean_attention, 'o-',
               color=color, linewidth=2, markersize=5,
               label=f'Layer {layer_idx + 1}', alpha=0.8)
        ax.fill_between(key_frame_indices,
                       mean_attention - std_attention,
                       mean_attention + std_attention,
                       color=color, alpha=0.2)

    # 英文标签
    ax.set_xlabel('Key Frame Index', fontsize=12)
    ax.set_ylabel('Mean Attention Weight', fontsize=12)
    ax.set_title('Cross-Layer Head-Averaged Attention Distribution',
                fontsize=14, fontweight='bold')
    ax.legend(loc='upper right', fontsize=10)
    ax.grid(True, alpha=0.3, linestyle='--')

    plt.tight_layout()

    # 保存为高质量 SVG
    if save_path:
        svg_path = save_path if save_path.endswith('.svg') else save_path.replace('.png', '.svg')
        plt.savefig(svg_path, format='svg', bbox_inches='tight',
                    metadata={'Creator': 'Self-Forcing Figure 4 Reproduction'})
        print(f"已保存到 {svg_path}")

    plt.show()
    return fig

In [ ]:
# 跨层注意力分布对比
fig = plot_attention_comparison(
    layer_attention_data,
    layers_to_compare=sorted(layer_attention_data.keys()),
    save_path="figure4_layer_comparison.svg"
)

## 6. 统计分析

验证论文的核心发现：注意力分布在整个上下文窗口中，而不仅仅集中在初始帧。

In [ ]:
def plot_attention_statistics(layer_attention_data: dict, save_path: str = None):
    """
    输出注意力分布的统计信息并可视化。
    验证论文的核心发现：注意力分布在整个上下文窗口中。
    图中 label 全部使用英文。
    """
    print("=" * 70)
    print("注意力分析摘要 (Figure 4 复现)")
    print("=" * 70)

    # 收集统计数据
    stats_data = []

    for layer_idx in sorted(layer_attention_data.keys()):
        data = layer_attention_data[layer_idx]
        frame_attention = data['frame_attention']
        key_frame_indices = data['key_frame_indices']
        num_frames = len(key_frame_indices)

        if num_frames < 3:
            continue

        # Head 平均注意力
        mean_attn = frame_attention.mean(axis=0)

        # 统计量
        first_frame_attn = mean_attn[0]
        last_frame_attn = mean_attn[-1]
        middle_attn = mean_attn[1:-1].mean() if len(mean_attn) > 2 else mean_attn.mean()
        max_frame = np.argmax(mean_attn)
        min_frame = np.argmin(mean_attn)

        stats_data.append({
            'layer': layer_idx + 1,
            'first': first_frame_attn,
            'middle': middle_attn,
            'last': last_frame_attn,
            'max_frame': max_frame,
            'min_frame': min_frame,
            'ratio_first_middle': first_frame_attn / middle_attn if middle_attn > 0 else 0,
        })

        print(f"\nLayer {layer_idx + 1} (Key 帧数: {num_frames}):")
        print(f"  首帧注意力:   {first_frame_attn:.6f}")
        print(f"  中间帧注意力: {middle_attn:.6f}")
        print(f"  末帧注意力:   {last_frame_attn:.6f}")
        print(f"  最大注意力帧: {max_frame}")
        print(f"  首帧/中间比:  {first_frame_attn/middle_attn:.2f}x" if middle_attn > 0 else "  首帧/中间比: N/A")

    print("\n" + "=" * 70)
    print("论文的关键发现：")
    print("  '大多数注意力头不仅对最早的 token 分配了大量权重，")
    print("   还对序列中间部分分配了相当的注意力。'")
    print("=" * 70)

    # 绘制统计可视化
    if len(stats_data) > 0:
        fig, ax = plt.subplots(figsize=(10, 5))

        layers = [s['layer'] for s in stats_data]
        first_attns = [s['first'] for s in stats_data]
        middle_attns = [s['middle'] for s in stats_data]
        last_attns = [s['last'] for s in stats_data]

        x = np.arange(len(layers))
        width = 0.25

        ax.bar(x - width, first_attns, width, label='First Frame', color='#2E86AB')
        ax.bar(x, middle_attns, width, label='Middle Frames', color='#F18F01')
        ax.bar(x + width, last_attns, width, label='Last Frame', color='#A23B72')

        # 英文标签
        ax.set_xlabel('Layer', fontsize=12)
        ax.set_ylabel('Mean Attention Weight', fontsize=12)
        ax.set_title('Attention Distribution: First vs Middle vs Last Frame',
                    fontsize=14, fontweight='bold')
        ax.set_xticks(x)
        ax.set_xticklabels([f'L{l}' for l in layers])
        ax.legend(fontsize=10)
        ax.grid(True, alpha=0.3, axis='y')

        plt.tight_layout()

        # 保存为高质量 SVG
        if save_path:
            svg_path = save_path if save_path.endswith('.svg') else save_path.replace('.png', '.svg')
            plt.savefig(svg_path, format='svg', bbox_inches='tight',
                        metadata={'Creator': 'Self-Forcing Figure 4 Reproduction'})
            print(f"\n已保存到 {svg_path}")

        plt.show()
        return fig

    return None

In [ ]:
# 生成统计分析和可视化
fig = plot_attention_statistics(layer_attention_data, save_path="figure4_statistics.svg")

## 7. 结果汇总

列出所有生成的高质量 SVG 图像文件。

In [ ]:
print("=" * 70)
print("FIGURE 4 复现完成")
print("=" * 70)
print(f"\nPrompt: {data.get('prompt', 'N/A')}")
print(f"帧数: {data.get('num_frames', 'N/A')}")
print(f"每 block 帧数: {data.get('num_frame_per_block', 'N/A')}")
print(f"\n捕获的层: {list(layer_attention_data.keys())}")

print("\n" + "-" * 70)
print("生成的高质量 SVG 文件（图中 label 均为英文）：")
print("-" * 70)
print("  - figure4_reproduction.svg     : 主 Figure 4 (L1H1 和 L5H10)")
print("  - figure4_layer1_all_heads.svg : Layer 1 所有 head 热力图")
print("  - figure4_layer5_all_heads.svg : Layer 5 所有 head 热力图")
print("  - figure4_layer_comparison.svg : 跨层对比")
print("  - figure4_statistics.svg       : 统计摘要")
print("-" * 70)

print("\n" + "=" * 70)
print("论文的关键观察：")
print("=" * 70)
print("""
'与传统理解（认为只需保留少量初始 KV token）不同，我们的分析揭示：
大多数注意力头不仅对最早的 token 分配了大量权重，还对序列中间部分
分配了相当的注意力。'

这一发现表明：对于自回归视频生成，需要保留完整的上下文，而不是像
LLM 中常用的 attention sink 机制那样只保留初始 token 和最近 token。
""")

## 8. 技术说明

### Figure 4 结果解读

**论文的关键发现：**
> "与传统理解（认为只需保留少量初始 KV token）不同，我们的分析揭示：
> 大多数注意力头不仅对最早的 token 分配了大量权重，还对序列中间部分分配了相当的注意力。"

### 这意味着什么：

1. **不仅仅是 Attention Sink**：与 LLM 中注意力通常集中在初始 token（attention sink 模式）不同，
   视频扩散模型的注意力在所有帧之间分布更均匀。

2. **完整上下文很重要**：对于自回归视频生成，保留完整的 KV cache 很重要——
   不仅仅是前几帧和最近的帧。

3. **层间差异**：不同层可能表现出不同的注意力模式。

### 技术细节：

- **Query**：最后一个 chunk（21 帧视频中的第 19-21 帧）
- **Key**：之前的 KV cache 条目（第 0-18 帧）
- **注意力形状**：`[B, num_heads, Lq, Lk]`
- **每帧 token 数**：1560 tokens（60×104 / 4 patches）

### SVG 输出配置：

- `svg.fonttype: none` - 使用真实字体而非路径，确保文字可编辑
- `savefig.dpi: 300` - 高分辨率输出
- 所有图中 label 使用英文，确保跨平台兼容性

In [ ]:
# 空单元格 - notebook 结束